In [1]:
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
import wandb
from fastai.callback.wandb import *
import numpy as np
from wandb_creds import wandb_api

In [3]:
wandb.login(key=wandb_api)

wandb: Currently logged in as: mkingo (feedback_prize_michael_and_wilson). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/noone/.netrc


True

In [4]:
wandb.init(project='KMNIST')

wandb: Currently logged in as: michael_kingston. Use `wandb login --relogin` to force relogin


In [5]:
wandb.init()

In [ ]:
! git clone https://github.com/rois-codh/kmnist.git

In [4]:
import os
os.chdir('kmnist')

^C
Traceback (most recent call last):
  File "/home/noone/Documents/GitHub/neural_networks/sandbox/kmnist/download_data.py", line 80, in <module>
    traverse_dict(download_dict)
  File "/home/noone/Documents/GitHub/neural_networks/sandbox/kmnist/download_data.py", line 63, in traverse_dict
    userinput = input('> ').strip()
KeyboardInterrupt


In [ ]:
# ! python download_data.py

# ! python download_data.py

In [9]:
train_imgs = np.load('kmnist-train-imgs.npz')['arr_0']#.reshape((232365, 1, 28, 28))
train_labels = np.int64(np.load('kmnist-train-labels.npz')['arr_0'])

train_imgs = np.expand_dims(train_imgs,axis=1)
train_imgs = np.float32(np.repeat(train_imgs, 3, axis=1))

test_imgs = np.load('kmnist-test-imgs.npz')['arr_0']#.reshape((38547, 1, 28, 28))
test_labels = np.int64(np.load('kmnist-test-labels.npz')['arr_0'])

test_imgs = np.expand_dims(test_imgs,axis=1)
test_imgs = np.float32(np.repeat(test_imgs, 3, axis=1))

FileNotFoundError: [Errno 2] No such file or directory: 'kmnist-train-imgs.npz'

In [ ]:
class NumpyArrayDataset(Dataset):

    def __init__(self, x, y):
        self.x, self.y = x, y
        self.c = np.unique(y).size # binary label
        self.classes = ["お", "き", "す", "つ", "な", "は", "ま", "や", "れ", "を"]

    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return self.x[i], self.y[i]

In [ ]:
train_ds = NumpyArrayDataset(train_imgs, train_labels)
valid_ds = NumpyArrayDataset(test_imgs, test_labels)

data = ImageDataBunch.create(train_ds, valid_ds, bs=128)
data.normalize(imagenet_stats)

In [ ]:
def kmnist_val_acc():
    metrics.accuracy

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)
learn.loss_func = torch.nn.functional.cross_entropy
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
%%wandb
learn.fit_one_cycle(10, max_lr=1e-2)